## 確認收斂次數

In [1]:
import unittest
from time import time, ctime

# CONSTANT
DATA_ROOT = "./data"
DBLP_PATH = DATA_ROOT + "/dblp"
AMAZON_PATH = DATA_ROOT + "/amazon"
FACEBOOK_PATH = DATA_ROOT + "/facebook"
NOTIFY_ENDPOINT = r"https://notify.run/O6EfLmG6Tof1s5DljYB7"


import pandas as pd

from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon

NUM_TOPICS = 5
TOPICS = {
    "Node": {
        "0": [0.9, 0.1, 0.0],
        "1": [0.2, 0.8, 0.0],
        "2": [0.8, 0.2, 0.0],
        "3": [0.2, 0.4, 0.4],
    },
    "Item": {
        "iPhone": [0.7, 0.0, 0.3],
        "AirPods": [0.9, 0.0, 0.1],
        "Galaxy": [0.0, 0.8, 0.2],
    }
}
PRICES = {
    "iPhone": 50,
    "AirPods": 5,
    "Galaxy": 60,
}
RELATION = pd.DataFrame.from_dict({
            "iPhone":{
                "AirPods":10,
                "Galaxy":-5
            },
            "AirPods":{
                "iPhone":1,
                "Galaxy":0,
            },
            "Galaxy":{
                "iPhone":-8,
                "AirPods":1,
            }
            })

In [2]:
items = read_items(AMAZON_PATH + "/sample_items.csv")

topicModel = TopicModel(NUM_TOPICS)
topicModel.read_topics(node_file=FACEBOOK_PATH + "/nodes_with_" + str(NUM_TOPICS) + "_topic.csv",
                        items_file=AMAZON_PATH + "/items_with_" + str(NUM_TOPICS) + "_topic.csv")

graph = SN_Graph.construct(FACEBOOK_PATH + "/edges", topicModel, located=False)

relation = ItemRelation()
relation.construct(AMAZON_PATH + "/sample_items.csv")
itemset = ItemsetFlyweight(getItemsPrice(AMAZON_PATH + "/sample_items.csv"), topicModel, relation)

model = DiffusionModel("amazon in dblp", graph, itemset, threshold=0)
seed_size = min(itemset.size, graph.number_of_nodes())
seeds = model.selectSeeds(seed_size)
model.allocate(seeds, [itemset[asin] for asin in itemset.PRICE.keys()])

simluation_times = 1
algo = Algorithm(model, 0, simulationTimes=simluation_times)
candidatedCoupons = [Coupon(20, itemset["B000JX5JGI B000VYIUJS"], 5, itemset["B000JX5JGI B000VYIUJS"])]

performanceFile = r"./result/distribution.txt"


for times in range(0,1):
    k=1
    algo.setLimitCoupon(k)
    model.diffusion()

    # with open(performanceFile, "a") as record:
        
    #     record.write("{0},runtime={1},revenue={2},expected_revenue={3},active_node={4},expected_active_node={5},times={6}\n".format(
    #         ctime(end_time),
    #         (end_time - start_time),
    #         tagger["TagRevenue"].amount(),
    #         tagger["TagRevenue"].expected_amount(),
    #         tagger["TagActiveNode"].amount(),
    #         tagger["TagActiveNode"].expected_amount(),
    #         times,
    #         ))
        
    #     for c in outputCoupons:
            
    #         record.write(str(c) + "\n")
    #     record.write("\n")
        

Constructing graph...
Connecting the edges...Done


In [ ]:
for u, attr in graph.nodes(data=True):
    if not attr["adopted_set"]:
        print("{}: ")

In [ ]:
length, path = SN_Graph.max_product_path(graph, model.getSeeds())

In [ ]:
bucket = dict()
for node, p in path.items():
    src = p[0]
    if src not in bucket:
        bucket[src] = dict()
    
    itemset = str(algo._graph.nodes[node]["adopted_set"])
    if itemset not in bucket[src]:
        bucket[src][itemset] = 1
    else:
        bucket[src][itemset] += 1

In [ ]:
seeds = algo._model.getSeeds()
for seed in seeds:
    graph = algo._model.getGraph()
    print("{0}: {1}".format("seed", seed))
    for k,v in graph.nodes[seed].items():
        print("{0}: {1}".format(k,v))
    print()

In [ ]:
import numpy as np
colors = {"B000VYIUJS":"64e9c0", "B000JX5JGI": "cbb446", "B000JX5JGI B000VYIUJS":"ba9800"}
for k,l in tagger["TagDecidedMainItemset"].items():
    print(k)
    print(l)
    # colors[k] = ((np.random.random(), np.random.random(), np.random.random()))

In [ ]:
import matplotlib.pyplot as plt

plt.figure("Main Itemset")
for k,v in tagger["TagDecidedMainItemset"].items():
    plt.plot(list(range(len(v))), v, 'ko--', color=colors[k], label=k, markerfacecolor='none')
plt.title("Withput coupons")
plt.xlabel("step")
plt.ylabel("times of decied main itemset")
plt.legend()

In [ ]:
str(outputCoupons[0])

### 可累積商品和累積門檻條件的變化對於使用者CP值感知的影響

In [7]:
import pandas as pd

from package.model import DiffusionModel
from package.topic import TopicModel
from package.social_graph import SN_Graph
from package.itemset import ItemsetFlyweight, ItemRelation
from package.utils import getItemsPrice, read_items
from package.algorithm import Algorithm
from package.coupon import Coupon
from package.user_proxy import UsersProxy

NUM_TOPICS = 5
TOPICS = {
    "Node": {
        "0": [0.9, 0.1, 0.0],
        "1": [0.2, 0.8, 0.0],
        "2": [0.8, 0.2, 0.0],
        "3": [0.2, 0.4, 0.4],
    },
    "Item": {
        "iPhone": [0.7, 0.0, 0.3],
        "AirPods": [0.9, 0.0, 0.1],
        "Galaxy": [0.0, 0.8, 0.2],
    }
}
PRICES = {
    "iPhone": 260.0,
    "AirPods": 60.0,
    "Galaxy": 500.0,
}
RELATION = pd.DataFrame.from_dict({
            "iPhone":{
                "AirPods":10,
                "Galaxy":-5
            },
            "AirPods":{
                "iPhone":1,
                "Galaxy":0,
            },
            "Galaxy":{
                "iPhone":-8,
                "AirPods":1,
            }
            })

topicModel = TopicModel(3, TOPICS["Node"], TOPICS["Item"])
relation = ItemRelation(RELATION)
itemset = ItemsetFlyweight(prices = PRICES, topic = topicModel.getItemsTopic(), relation=relation)
graph = SN_Graph(topicModel.getNodesTopic())
coupon = Coupon(500, itemset["Galaxy"], itemset["AirPods iPhone"].price*0.6, itemset["AirPods iPhone"])
user_proxy = UsersProxy(graph, itemset, [coupon], 0)
user_id = "0"
main_itemset = itemset["Galaxy"]
graph.add_node(user_id, adopted_set=None, desired_set=main_itemset, topic=TOPICS["Node"][user_id])

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

step = 20
# 可累積商品
list_accItemset = [id for id, items in itemset if itemset.issubset(main_itemset, items)]

# 累積門檻
list_accThreshold = list(np.arange(main_itemset.price, itemset["Galaxy iPhone AirPods"].price+step*10, step))

# 折抵商品
list_disItemset = [id for id, items in itemset]

# 折抵金額
list_discount = list(np.arange(0, itemset["Galaxy iPhone AirPods"].price, step=20))

multi_index = []
data = []
results = [] # for debug

for acc_itemset_id in list_accItemset:
    coupon.accItemset = itemset[acc_itemset_id]

    for threshold in list_accThreshold:
        coupon.accThreshold = threshold

        for dis_itemset_id in list_disItemset:
            coupon.disItemset = itemset[dis_itemset_id]

            for discount in list_discount:
                coupon.discount = discount

                user_proxy.setCoupons([coupon])
            
                result = user_proxy.adopt(user_id)
                results.append(result)

                # reset
                graph._initNode(user_id)
                graph.nodes[user_id]["desired_set"] = main_itemset
                
                multi_index.append((acc_itemset_id, threshold, dis_itemset_id, discount))
                data.append([result["VP"], result["amount"], result["tradeOff_items"]])


In [3]:
# create a multiple index dataframe
index = pd.MultiIndex.from_tuples(multi_index, names=('可累積商品', '累積門檻', '折抵商品', '折抵金額'))
df = pd.DataFrame(data=data ,index=index, columns=['VP值', '價格', '購買商品組合'])

In [4]:
pd.set_option('display.max_columns', None)
# df = df.reorder_levels(['可累積商品', '累積門檻', '折抵金額', '折抵商品']).sort_index(level=['可累積商品', '累積門檻', '折抵金額', '折抵商品'])

In [13]:
coupon_test = Coupon(501, itemset["AirPods Galaxy iPhone"], 40, itemset["AirPods Galaxy iPhone"])
graph._initNode(user_id)
print("Main itemset: {0}".format(user_proxy._mainItemsetVP(user_id, main_itemset)))

for id, obj in itemset:
    if itemset.issubset(main_itemset, obj):
        VP = user_proxy._addtionallyAdoptVP(user_id, main_itemset, obj, coupon_test)
        print("{0}: {1:.8f}".format(id, VP))

Main itemset: 0.00016000000000000004
Galaxy: 0.00015968
Galaxy iPhone: 0.00048250
AirPods Galaxy: 0.00085985
AirPods Galaxy iPhone: 0.00066391


In [14]:
coupon_test = Coupon(561, itemset["AirPods Galaxy iPhone"], 60, itemset["AirPods Galaxy iPhone"])
graph._initNode(user_id)
print("Main itemset: {0}".format(user_proxy._mainItemsetVP(user_id, main_itemset)))

for id, obj in itemset:
    if itemset.issubset(main_itemset, obj):
        VP = user_proxy._addtionallyAdoptVP(user_id, main_itemset, obj, coupon_test)
        print("{0}: {1:.8f}".format(id, VP))

Main itemset: 0.00016000000000000004
Galaxy: 0.00014260
Galaxy iPhone: 0.00044320
AirPods Galaxy: 0.00071303
AirPods Galaxy iPhone: 0.00060850


In [ ]:
"{0:.4E}".format(user_proxy._mainItemsetVP(user_id, main_itemset))

In [ ]:
df = df.reorder_levels(['可累積商品', '累積門檻', '折抵商品', '折抵金額'])

In [ ]:
df.loc[(df['價格'] > main_itemset.price) & (df['VP值'] > 0.002423076923076923)]

In [ ]:
df.loc[("AirPods Galaxy iPhone", 280, "AirPods")]

1. 將折抵商品分為兩組，有無包含主商品
2. 分成兩個角度觀察
    - 可累積商品為同一組，觀察不同折抵商品組合間，在不同折抵金額下隨著累積門檻越高會有什麼變化
    - 反過來觀察可折抵商品一組

In [ ]:
# 折抵商品不包含主商品
dis_items_exclued_main_items = df[~df.index.get_level_values("折抵商品").str.contains("Galaxy")]
print(dis_items_exclued_main_items)

In [ ]:
# 可累積商品
index = 0
acc_itemset = list_accItemset[index]
select_acc_itemset = dis_items_exclued_main_items[dis_items_exclued_main_items.index.get_level_values("可累積商品") == acc_itemset]
print(list_accItemset)
print(list_accItemset[index])


In [ ]:
colors = ["#242E91", "#91233B", "#23913D", "#ff3399"]
markers = ["o", "v", "s", "p", "x"]

In [ ]:
set(select_acc_itemset.index.get_level_values('折抵商品'))

In [ ]:
pd.set_option('display.max_rows', None)

for distItemset_id in set(select_acc_itemset.index.get_level_values('折抵商品')): # 折抵商品組合
    select_dis_itemset = select_acc_itemset[select_acc_itemset.index.get_level_values("折抵商品") == distItemset_id]
    print(select_dis_itemset)
    # for i in range(len(list_discount)): # 折抵金額
    #     select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
    #     print(select_discount)

In [ ]:
for distItemset_id in set(select_acc_itemset.index.get_level_values('折抵商品')): # 折抵商品組合
    select_dis_itemset = select_acc_itemset[select_acc_itemset.index.get_level_values("折抵商品") == distItemset_id]
    print(distItemset_id)
    plt.figure("the relation of VP with discount items: {0}".format(distItemset_id))
    for i in range(len(list_discount)): # 折抵金額
        select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
        print(list_discount[i])
        plt.plot(list_accThreshold, list(select_discount["VP值"]), 'k' + markers[i] + '--', 
                color=colors[i], 
                label="{:.0f}% off".format(list_discount[i]*100), 
                markerfacecolor='none')
    plt.title("the relation of VP with discount items: {0}".format(distItemset_id))
    plt.xlabel("Threshold of coupon")
    plt.ylabel("VP")
    plt.legend()

    plt.figure("the relation of amount with discount items: {0}".format(distItemset_id))
    for i in range(len(list_discount)): # 折抵金額
        select_discount = select_dis_itemset[select_dis_itemset.index.get_level_values("折抵金額") == list_discount[i]]
        plt.plot(list_accThreshold, list(select_discount["價格"]), 'k' + markers[i] + '--', 
                color=colors[i], 
                label="{:.0f}% off".format(list_discount[i]*100), 
                markerfacecolor='none')
    plt.title("the relation of amount with discount items: {0}".format(distItemset_id))
    plt.xlabel("Threshold of coupon")
    plt.ylabel("VP")
    plt.legend()